<a href="https://colab.research.google.com/github/ripper763/CS29006_SW_Lab_Spr2023/blob/main/amex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.7 MB/s eta 0:00:00


Importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
train_data = pd.read_csv('663e2b6d54457_train_data_with_samplefeatures.csv')
test_data = pd.read_csv('667a986f0b981_r2_data_with_samplefeatures.csv')
batsman_scorecard  = pd.read_csv('667a985a7541a_batsman_level_scorecard.csv')
bowler_scorecard  = pd.read_csv('667a98513f849_bowler_level_scorecard.csv')
match_scorecard  = pd.read_csv('667a9888c9104_match_level_scorecard.csv')

Not violating the rules of the game/ merging all datasets

In [ ]:
for df in [train_data, test_data, batsman_scorecard, bowler_scorecard, match_scorecard]:
    df['match_dt'] = pd.to_datetime(df['match_dt'])
    df.sort_values('match_dt', inplace=True)

In [ ]:
def get_historical_data(df, current_date, window=15):
    return df[df['match_dt'] < current_date].tail(window)

In [ ]:

# Aggregate batsman data
batsman_agg = batsman_scorecard.groupby('match id').agg({
    'runs': 'sum',
    'balls_faced': 'sum',
    'strike_rate': 'mean',
    'Fours': 'sum',
    'Sixes': 'sum'
}).add_prefix('batsman_')

# Aggregate bowler data
bowler_agg = bowler_scorecard.groupby('match id').agg({
    'runs': 'sum',
    'wicket_count': 'sum',
    'balls_bowled': 'sum',
    'economy': 'mean',
    'maiden': 'sum',
    'dots': 'sum',
    'Fours': 'sum',
    'Sixes': 'sum',
    'wides': 'sum',
    'noballs': 'sum'
}).add_prefix('bowler_')

# Merge for train data
train_merged = pd.merge(train_data, batsman_agg, left_on='match id', right_index=True, how='left')
train_merged = pd.merge(train_merged, bowler_agg, left_on='match id', right_index=True, how='left')
train_merged = pd.merge(train_merged, match_scorecard, on='match id', how='left')

# Merge for test data
test_merged = pd.merge(test_data, batsman_agg, left_on='match id', right_index=True, how='left')
test_merged = pd.merge(test_merged, bowler_agg, left_on='match id', right_index=True, how='left')
test_merged = pd.merge(test_merged, match_scorecard, on='match id', how='left')

# Save merged dat

In [ ]:
for df in [train_data, test_data, batsman_scorecard, bowler_scorecard, match_scorecard]:
    df['match_dt'] = pd.to_datetime(df['match_dt'])
    df.sort_values('match_dt', inplace=True)

def get_historical_data(df, current_date, team1=None, team2=None, ground=None, window=15):
    mask = pd.Series(True, index=df.index)
    if team1:
        mask &= (df['team1_x'] == team1) | (df['team2_x'] == team1)
    if team2:
        mask &= (df['team1_x'] == team2) | (df['team2_x'] == team2)
    if ground:
        mask &= (df['ground_id_x'] == ground)

    historical_data = df[mask & (df['match_dt_x'] < current_date)].tail(window)
    return historical_data

def create_historical_features(df):
    df['team_count_50runs'] = df.apply(lambda row: get_historical_data(df, row['match_dt_x'], team1=row['team1_x'], team2=row['team2_x'])['team_count_50runs_last15'].mean() if len(get_historical_data(df, row['match_dt_x'], team1=row['team1_x'], team2=row['team2_x'])) > 0 else 0, axis=1)
    df['team_winp'] = df.apply(lambda row: get_historical_data(df, row['match_dt_x'], team1=row['team1_x'], team2=row['team2_x'], window=5)['team_winp_last5'].mean() if len(get_historical_data(df, row['match_dt_x'], team1=row['team1_x'], team2=row['team2_x'], window=5)) > 0 else 0, axis=1)
    df['team1only_avg_runs'] = df.apply(lambda row: get_historical_data(df, row['match_dt_x'], team1=row['team1_x'])['team1only_avg_runs_last15'].mean() if len(get_historical_data(df, row['match_dt_x'], team1=row['team1_x'])) > 0 else 0, axis=1)
    df['team1_winp_team2'] = df.apply(lambda row: get_historical_data(df, row['match_dt_x'], team1=row['team1_x'], team2=row['team2_x'])['team1_winp_team2_last15'].mean() if len(get_historical_data(df, row['match_dt_x'], team1=row['team1_x'], team2=row['team2_x'])) > 0 else 0, axis=1)
    df['ground_avg_runs'] = df.apply(lambda row: get_historical_data(df, row['match_dt_x'], ground=row['ground_id_x'])['ground_avg_runs_last15'].mean() if len(get_historical_data(df, row['match_dt_x'], ground=row['ground_id_x'])) > 0 else 0, axis=1)
    return df

train_merged = create_historical_features(train_merged)
test_merged = create_historical_features(test_merged)

Columns of merged train dataset

In [ ]:
print(test_merged.columns)

Index(['match id', 'team1_x', 'team1_id_x', 'team1_roster_ids_x', 'team2_x',
       'team2_id_x', 'team2_roster_ids_x', 'toss winner_x', 'toss decision_x',
       'venue_x', 'city_x', 'match_dt_x', 'lighting_x', 'series_name_x',
       'season_x', 'ground_id_x', 'team_count_50runs_last15',
       'team_winp_last5', 'team1only_avg_runs_last15',
       'team1_winp_team2_last15', 'ground_avg_runs_last15', 'batsman_runs',
       'batsman_balls_faced', 'batsman_strike_rate', 'batsman_Fours',
       'batsman_Sixes', 'bowler_runs', 'bowler_wicket_count',
       'bowler_balls_bowled', 'bowler_economy', 'bowler_maiden', 'bowler_dots',
       'bowler_Fours', 'bowler_Sixes', 'bowler_wides', 'bowler_noballs',
       'team1_y', 'team2_y', 'winner', 'by', 'win amount', 'toss winner_y',
       'toss decision_y', 'venue_y', 'city_y', 'match_dt_y', 'lighting_y',
       'series_name_y', 'season_y', 'ground_id_y', 'umpire1', 'umpire2',
       'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_r

'team_count_50runs', 'team_winp', 'team1only_avg_runs',
       'team1_winp_team2', 'ground_avg_runs', 'team1_top_3_batsmen_avg_sr',
       'team2_top_3_batsmen_avg_sr', 'team1_top_3_bowlers_avg_econ',
       'team2_top_3_bowlers_avg_econ', 'ratio_top_3batsmen_avg_sr',
       'ratio_top_3_bowlers_avg_econ', 'toss_winner_01', 'toss_decision_01',
       'venue_toss_advantage_field', 'venue_toss_advantage_bat',
       'team1_win_percentage', 'team2_win_percentage',
       'team1_head_to_head', 'team2_head_to_head',
       'head_to_head_win_percentage', 'ratio_win_percentage'

In [ ]:
print(train_merged["avg_wickets_at_venue_team2"])

KeyError: 'avg_wickets_at_venue_team2'

Feature engineering

In [ ]:
# Top 3 Batsmen Strike Rate
def get_top_3_batsmen_sr(group):
    # Sort batsmen by runs scored in descending order
    sorted_batsmen = group.sort_values('runs', ascending=False)

    # Get the top 3 batsmen
    top_3 = sorted_batsmen.head(3)

    # Calculate average strike rate
    return top_3['strike_rate'].mean()

# Group by match and inning to get top 3 batsmen for each team in each match
batsman_team_sr = batsman_scorecard.groupby(['match id', 'inning']).apply(get_top_3_batsmen_sr).reset_index()
batsman_team_sr.columns = ['match id', 'inning', 'top_3_batsmen_avg_sr']

# Split into team1 and team2 features
team1_batsmen_sr = batsman_team_sr[batsman_team_sr['inning'] == 1].drop('inning', axis=1)
team1_batsmen_sr.columns = ['match id', 'team1_top_3_batsmen_avg_sr']

team2_batsmen_sr = batsman_team_sr[batsman_team_sr['inning'] == 2].drop('inning', axis=1)
team2_batsmen_sr.columns = ['match id', 'team2_top_3_batsmen_avg_sr']

# Merge with train and test data
train_merged = train_merged.merge(team1_batsmen_sr, on='match id', how='left')
train_merged = train_merged.merge(team2_batsmen_sr, on='match id', how='left')
test_merged = test_merged.merge(team1_batsmen_sr, on='match id', how='left')
test_merged = test_merged.merge(team2_batsmen_sr, on='match id', how='left')

# Top 3 Bowlers Economy Rate
def get_top_3_bowlers_econ(group):
    # Sort bowlers by wickets taken in descending order, then by economy rate in ascending order
    sorted_bowlers = group.sort_values(['wicket_count', 'economy'], ascending=[False, True])

    # Get the top 3 bowlers
    top_3 = sorted_bowlers.head(3)

    # Calculate average economy rate
    return top_3['economy'].mean()

# Group by match and inning to get top 3 bowlers for each team in each match
bowler_team_econ = bowler_scorecard.groupby(['match id', 'inning']).apply(get_top_3_bowlers_econ).reset_index()
bowler_team_econ.columns = ['match id', 'inning', 'top_3_bowlers_avg_econ']

# Split into team1 and team2 features
team1_bowlers_econ = bowler_team_econ[bowler_team_econ['inning'] == 1].drop('inning', axis=1)
team1_bowlers_econ.columns = ['match id', 'team1_top_3_bowlers_avg_econ']

team2_bowlers_econ = bowler_team_econ[bowler_team_econ['inning'] == 2].drop('inning', axis=1)
team2_bowlers_econ.columns = ['match id', 'team2_top_3_bowlers_avg_econ']

# Merge with train and test data
train_merged = train_merged.merge(team1_bowlers_econ, on='match id', how='left')
train_merged = train_merged.merge(team2_bowlers_econ, on='match id', how='left')
test_merged = test_merged.merge(team1_bowlers_econ, on='match id', how='left')
test_merged = test_merged.merge(team2_bowlers_econ, on='match id', how='left')

In [ ]:
train_merged['ratio_top_3batsmen_avg_sr'] = train_merged['team1_top_3_batsmen_avg_sr'] / train_merged['team2_top_3_batsmen_avg_sr']
train_merged['ratio_top_3_bowlers_avg_econ'] = train_merged['team1_top_3_bowlers_avg_econ'] / train_merged['team2_top_3_bowlers_avg_econ']

test_merged['ratio_top_3batsmen_avg_sr'] = test_merged['team1_top_3_batsmen_avg_sr'] / test_merged['team2_top_3_batsmen_avg_sr']
test_merged['ratio_top_3_bowlers_avg_econ'] = test_merged['team1_top_3_bowlers_avg_econ'] / test_merged['team2_top_3_bowlers_avg_econ']

In [ ]:
## Toss winner to numerical - 1 if team2 wins, else 0
train_merged['toss_winner_01'] = np.where(train_merged['toss winner_x']==train_merged['team2_x'], 1, 0)
test_merged['toss_winner_01'] = np.where(test_merged['toss winner_x']==test_merged['team2_x'], 1, 0)

## Toss decision - categorical - 1 if winner bats, 0 otherwise
train_merged['toss_decision_01'] = np.where(train_merged['toss decision_x']=='bat', 1, 0)
test_merged['toss_decision_01'] = np.where(test_merged['toss decision_x']=='bat', 1, 0)

In [ ]:
# Calculate venue-specific toss advantage considering toss decision
venue_toss_adv = train_merged.groupby(['venue_x', 'toss_decision_01'])['toss_winner_01'].mean().reset_index()
venue_toss_adv = venue_toss_adv.pivot_table(index='venue_x', columns='toss_decision_01', values='toss_winner_01', fill_value=0)
venue_toss_adv.columns = ['venue_toss_advantage_field', 'venue_toss_advantage_bat']
venue_toss_adv.reset_index(inplace=True)

# Merge with train and test data
train_merged = train_merged.merge(venue_toss_adv, on='venue_x', how='left')
test_merged = test_merged.merge(venue_toss_adv, on='venue_x', how='left')

In [ ]:
# Create new columns in match_level_scorecard
match_scorecard['team1_won'] = np.where(match_scorecard['winner'] == match_scorecard['team1'], 1, 0)
match_scorecard['team2_won'] = np.where(match_scorecard['winner'] == match_scorecard['team2'], 1, 0)
match_scorecard['win_margin'] = match_scorecard['win amount'].abs()
match_scorecard['won_by'] = match_scorecard['by']

# Group by team1 and team2 to get historical win percentages
team1_wins = match_scorecard.groupby('team1')['team1_won'].mean().reset_index()
team2_wins = match_scorecard.groupby('team2')['team2_won'].mean().reset_index()

team1_wins.columns = ['team1', 'team1_win_percentage']
team2_wins.columns = ['team2', 'team2_win_percentage']

# Merge the team win percentages with train_merged
train_merged = train_merged.merge(team1_wins, left_on='team1_x', right_on='team1', how='left')
train_merged = train_merged.merge(team2_wins, left_on='team2_x', right_on='team2', how='left')

# Merge the team win percentages with test_merged
test_merged = test_merged.merge(team1_wins, left_on='team1_x', right_on='team1', how='left')
test_merged = test_merged.merge(team2_wins, left_on='team2_x', right_on='team2', how='left')

# Create a new column for head-to-head win percentage
head_to_head = match_scorecard.groupby(['team1', 'team2'])['team1_won'].mean().reset_index()
head_to_head.columns = ['team1', 'team2', 'head_to_head_win_percentage']

# Merge the head-to-head win percentage with train_merged and test_merged
train_merged = train_merged.merge(head_to_head, left_on=['team1_x', 'team2_x'], right_on=['team1', 'team2'], how='left', suffixes=('', '_head_to_head'))
test_merged = test_merged.merge(head_to_head, left_on=['team1_x', 'team2_x'], right_on=['team1', 'team2'], how='left', suffixes=('', '_head_to_head'))

In [ ]:
train_merged['ratio_win_percentage'] = train_merged['team1_win_percentage'] * train_merged['team2_win_percentage']
test_merged['ratio_win_percentage'] = test_merged['team1_win_percentage'] * test_merged['team2_win_percentage']

In [ ]:
train_merged['winner_01'] = train_merged.apply(lambda x: 1 if (x['team2']==x['winner_x']) else 0, axis=1)

In [ ]:
def create_ground_average_runs_feature(match_scorecard, train_data, test_data):
    # Calculate total runs for each match
    match_scorecard['total_runs'] = match_scorecard['inning1_runs'] + match_scorecard['inning2_runs']

    # Group by ground and calculate average runs
    ground_avg_runs = match_scorecard.groupby('ground_id')['total_runs'].mean()

    # Convert to DataFrame for easier merging
    ground_avg_runs = ground_avg_runs.reset_index()
    ground_avg_runs.columns = ['ground_id', 'ground_average_runs']

    # Merge with train data
    train_data = pd.merge(train_data, ground_avg_runs, left_on='ground_id_x', right_on='ground_id', how='left')

    # Merge with test data
    test_data = pd.merge(test_data, ground_avg_runs, left_on='ground_id_x', right_on='ground_id', how='left')

    # Fill NaN values with overall average (in case there are new grounds in test data)
    overall_average = ground_avg_runs['ground_average_runs'].mean()
    train_data['ground_average_runs'] = train_data['ground_average_runs'].fillna(overall_average)
    test_data['ground_average_runs'] = test_data['ground_average_runs'].fillna(overall_average)

    # Drop the redundant 'ground_id' column (from the right table in the merge)
    train_data = train_data.drop('ground_id', axis=1)
    test_data = test_data.drop('ground_id', axis=1)

    return train_data, test_data

# Usage
train_merged, test_merged = create_ground_average_runs_feature(match_scorecard, train_merged, test_merged)

# Verify the new feature
print(train_merged[['ground_id_x', 'ground_average_runs']].head())
print(test_merged[['ground_id_x', 'ground_average_runs']].head())

   ground_id_x  ground_average_runs
0         6348           300.000000
1         2932           310.000000
2          440           307.720000
3          930           315.521739
4         3961           274.285714
   ground_id_x  ground_average_runs
0         8252           318.578947
1          440           307.720000
2         1476           326.600000
3         8252           318.578947
4          930           315.521739


In [ ]:
# Create a feature for team win percentage in a series
def create_team_win_percentage_in_series(train_merged, test_merged, match_scorecard):
    # Get the unique series names
    series_names = match_scorecard['series_name'].unique()

    # Create an empty dictionary to store team win percentages for each series
    series_team_win_percentage = {}

    # Iterate over each series
    for series in series_names:
        # Filter the match_scorecard for the current series
        series_matches = match_scorecard[match_scorecard['series_name'] == series]

        # Calculate team1 and team2 win counts for the series
        team1_wins = series_matches['team1_won'].sum()
        team2_wins = series_matches['team2_won'].sum()
        total_matches = len(series_matches)

        # Calculate team1 and team2 win percentages for the series
        if total_matches > 0:
            team1_win_percentage = team1_wins / total_matches
            team2_win_percentage = team2_wins / total_matches
        else:
            team1_win_percentage = 0
            team2_win_percentage = 0

        # Store the win percentages in the dictionary
        series_team_win_percentage[series] = {
            'team1_win_percentage': team1_win_percentage,
            'team2_win_percentage': team2_win_percentage
        }

    # Merge the win percentage features with train and test data
    train_merged['team1_win_percentage_in_series'] = train_merged.apply(
        lambda row: series_team_win_percentage.get(row['series_name_x'], {}).get('team1_win_percentage', 0), axis=1
    )
    train_merged['team2_win_percentage_in_series'] = train_merged.apply(
        lambda row: series_team_win_percentage.get(row['series_name_x'], {}).get('team2_win_percentage', 0), axis=1
    )

    test_merged['team1_win_percentage_in_series'] = test_merged.apply(
        lambda row: series_team_win_percentage.get(row['series_name_x'], {}).get('team1_win_percentage', 0), axis=1
    )
    test_merged['team2_win_percentage_in_series'] = test_merged.apply(
        lambda row: series_team_win_percentage.get(row['series_name_x'], {}).get('team2_win_percentage', 0), axis=1
    )

    return train_merged, test_merged

# Call the function to create the new features
train_merged, test_merged = create_team_win_percentage_in_series(train_merged, test_merged, match_scorecard)

In [ ]:
train_merged['ratio_win_percentage_in_series'] = train_merged['team1_win_percentage_in_series'] * train_merged['team2_win_percentage_in_series']
test_merged['ratio_win_percentage_in_series'] = test_merged['team1_win_percentage_in_series'] * test_merged['team2_win_percentage_in_series']

In [ ]:
def calculate_recent_form(df, window=5):
    df = df.sort_values('match_dt_x')

    def calculate_team_form(team_col, opp_col):
        team_df = df[[team_col, 'match_dt_x', 'batsman_runs', 'bowler_wicket_count']].copy()
        team_df['team'] = team_df[team_col]
        team_df = team_df.sort_values(['team', 'match_dt_x'])

        team_df[f'{team_col}_last_{window}_runs'] = team_df.groupby('team')['batsman_runs'].transform(lambda x: x.shift().rolling(window, min_periods=1).mean())
        team_df[f'{team_col}_last_{window}_wickets'] = team_df.groupby('team')['bowler_wicket_count'].transform(lambda x: x.shift().rolling(window, min_periods=1).mean())

        return team_df[[team_col, 'match_dt_x', f'{team_col}_last_{window}_runs', f'{team_col}_last_{window}_wickets']]

    team1_form = calculate_team_form('team1_x', 'team2_x')
    team2_form = calculate_team_form('team2_x', 'team1_x')

    df = df.merge(team1_form, on=['team1_x', 'match_dt_x'], how='left')
    df = df.merge(team2_form, on=['team2_x', 'match_dt_x'], how='left')

    return df

# Apply the function to both train and test data
train_merged = calculate_recent_form(train_merged)
test_merged = calculate_recent_form(test_merged)

In [ ]:
train_merged['ratio_last_5_runs'] = train_merged['team1_x_last_5_runs'] * train_merged['team2_x_last_5_runs']
test_merged['ratio_last_5_runs'] = test_merged['team1_x_last_5_runs'] * test_merged['team2_x_last_5_runs']

train_merged['ratio_last_5_wickets'] = train_merged['team1_x_last_5_wickets'] * train_merged['team2_x_last_5_wickets']
test_merged['ratio_last_5_wickets'] = test_merged['team1_x_last_5_wickets'] * test_merged['team2_x_last_5_wickets']

In [ ]:
def calculate_venue_performance(df):
    venue_perf = df.groupby(['venue_x', 'team1_x']).agg({
        'batsman_runs': 'mean',
        'bowler_wicket_count': 'mean'
    }).reset_index()
    venue_perf.columns = ['venue_x', 'team', 'avg_runs_at_venue', 'avg_wickets_at_venue']
    return venue_perf

venue_performance = calculate_venue_performance(train_merged)

# Merge with train and test data
train_merged = train_merged.merge(venue_performance, left_on=['venue_x', 'team1_x'], right_on=['venue_x', 'team'], how='left')
train_merged = train_merged.merge(venue_performance, left_on=['venue_x', 'team2_x'], right_on=['venue_x', 'team'], how='left', suffixes=('_team1', '_team2'))
test_merged = test_merged.merge(venue_performance, left_on=['venue_x', 'team1_x'], right_on=['venue_x', 'team'], how='left')
test_merged = test_merged.merge(venue_performance, left_on=['venue_x', 'team2_x'], right_on=['venue_x', 'team'], how='left', suffixes=('_team1', '_team2'))

In [ ]:
train_merged['ratio_avg_runs_at_venue'] = train_merged['avg_runs_at_venue_team1'] / train_merged['avg_runs_at_venue_team2']
test_merged['ratio_avg_runs_at_venue'] = test_merged['avg_runs_at_venue_team1'] / test_merged['avg_runs_at_venue_team2']

train_merged['ratio_avg_wickets_at_venue'] = train_merged['avg_wickets_at_venue_team1'] / train_merged['avg_wickets_at_venue_team2']
test_merged['ratio_avg_wickets_at_venue'] = test_merged['avg_wickets_at_venue_team1'] / test_merged['avg_wickets_at_venue_team2']

Dividing data for training and cleaning

In [ ]:
print(train_merged.columns[-50:])

Index(['inning2_wickets', 'inning2_balls', 'team1_id_y', 'team1_roster_ids_y',
       'team2_id_y', 'team2_roster_ids_y', 'series_type', 'winner_id_y',
       'player_of_the_match_id', 'team_count_50runs', 'team_winp',
       'team1only_avg_runs', 'team1_winp_team2', 'ground_avg_runs',
       'team1_top_3_batsmen_avg_sr', 'team2_top_3_batsmen_avg_sr',
       'team1_top_3_bowlers_avg_econ', 'team2_top_3_bowlers_avg_econ',
       'ratio_top_3batsmen_avg_sr', 'ratio_top_3_bowlers_avg_econ',
       'toss_winner_01', 'toss_decision_01', 'venue_toss_advantage_field',
       'venue_toss_advantage_bat', 'team1', 'team1_win_percentage', 'team2',
       'team2_win_percentage', 'team1_head_to_head', 'team2_head_to_head',
       'head_to_head_win_percentage', 'ratio_win_percentage', 'winner_01',
       'team1_win_percentage_in_series', 'team2_win_percentage_in_series',
       'ratio_win_percentage_in_series', 'team1_x_last_5_runs',
       'team1_x_last_5_wickets', 'team2_x_last_5_runs',
       'te

In [ ]:
X,y = train_merged[['head_to_head_win_percentage',
 'ratio_top_3_bowlers_avg_econ',
'ratio_top_3batsmen_avg_sr',
'ground_avg_runs',
'team1only_avg_runs' ,
'ratio_last_5_wickets',
'ratio_win_percentage']], train_merged['winner_01']

0      176.00
1         NaN
2      234.00
3      178.00
4         NaN
        ...  
970       NaN
971       NaN
972       NaN
973    286.75
974    309.00
Name: avg_runs_at_venue_team2, Length: 975, dtype: float64


In [ ]:
X,y = train_merged[[ 'team_count_50runs', 'team_winp',
       'team1only_avg_runs', 'team1_winp_team2', 'ground_avg_runs',
      'ratio_top_3batsmen_avg_sr', 'ratio_top_3_bowlers_avg_econ',
                     'venue_toss_advantage_field',
       'venue_toss_advantage_bat','head_to_head_win_percentage', 'ratio_win_percentage',
                  'ratio_win_percentage_in_series',  'ratio_last_5_runs', 'ratio_last_5_wickets',
                     'ratio_avg_runs_at_venue', 'ratio_avg_wickets_at_venue']], train_merged['winner_01']

'team1_winp_team2', 'ground_avg_runs', 'ratio_top_3batsmen_avg_sr',
       'ratio_top_3_bowlers_avg_econ', 'head_to_head_win_percentage

```
# This is formatted as code
```



In [ ]:
X_test = test_merged[X.columns.tolist()]

In [ ]:
X_nans = X.isna().sum().reset_index()
X_nans[X_nans[0]!=0]

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans[X_test_nans[0]!=0]

,index,0
0,head_to_head_win_percentage,54
1,ratio_top_3_bowlers_avg_econ,207
2,ratio_top_3batsmen_avg_sr,207
3,ground_avg_runs,5
5,ratio_last_5_wickets,207
6,ratio_win_percentage,18


In [ ]:
# Replace NaNs in training data with mean of the respective feature
for feature in X.columns:
    X.loc[:, feature] = X[feature].fillna(X[feature].mean())

# Replace NaNs in test data with mean of the respective feature
for feature in X_test.columns:
    X_test.loc[:, feature] = X_test[feature].fillna(X_test[feature].mean())

# Check if there are any remaining NaNs
X_nans = X.isna().sum().reset_index()
X_nans = X_nans[X_nans[0] != 0]
print("NaNs remaining in training data:", X_nans)

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans = X_test_nans[X_test_nans[0] != 0]
print("NaNs remaining in test data:", X_test_nans)

ValueError: Setting with non-unique columns is not allowed.

In [ ]:
# Replace NaNs in ratio_top_3batsmen_avg_sr and ratio_top_3_bowlers_avg_econ with 1 for test data
X_test.loc[X_test['ratio_top_3batsmen_avg_sr'].isna(), 'ratio_top_3batsmen_avg_sr'] = 1
X_test.loc[X_test['ratio_top_3_bowlers_avg_econ'].isna(), 'ratio_top_3_bowlers_avg_econ'] = 1

In [ ]:
# Replace NaNs in training data with mean of the respective feature
for feature in X.columns:
    X.loc[:, feature] = X[feature].fillna(X[feature].mean())

# Replace NaNs in test data with mean of the respective feature
for feature in X_test.columns:
    feature_mean = X[feature].mean()  # Calculate mean from training data
    X_test.loc[:, feature] = X_test[feature].fillna(feature_mean)

# Check if there are any remaining NaNs
X_nans = X.isna().sum().reset_index()
X_nans = X_nans[X_nans[0] != 0]
print("NaNs remaining in training data:", X_nans)

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans = X_test_nans[X_test_nans[0] != 0]
print("NaNs remaining in test data:", X_test_nans)

NaNs remaining in training data: Empty DataFrame
Columns: [index, 0]
Index: []
NaNs remaining in test data: Empty DataFrame
Columns: [index, 0]
Index: []


In [ ]:
 # Replace NaNs in training data with mean of the respective feature
for feature in X.columns:
    X[feature].fillna(X[feature].mean(), inplace=True)

# Replace NaNs in test data with mean of the respective feature
for feature in X_test.columns:
    X_test[feature].fillna(X_test[feature].mean(), inplace=True)

# Check if there are any remaining NaNs
X_nans = X.isna().sum().reset_index()
X_nans = X_nans[X_nans[0] != 0]
print("NaNs remaining in training data:", X_nans)

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans = X_test_nans[X_test_nans[0] != 0]
print("NaNs remaining in test data:", X_test_nans)

NaNs remaining in training data: Empty DataFrame
Columns: [index, 0]
Index: []
NaNs remaining in test data:                           index    0
5     ratio_top_3batsmen_avg_sr  271
6  ratio_top_3_bowlers_avg_econ  271


<ipython-input-26-8eef1b897156>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature].fillna(X[feature].mean(), inplace=True)
<ipython-input-26-8eef1b897156>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature].fillna(X_test[feature].mean(), inplace=True)


In [ ]:
X.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

<ipython-input-30-bd600549b45e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)
<ipython-input-30-bd600549b45e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(0, inplace=True)


In [ ]:
X_nans = X.isna().sum().reset_index()
X_nans[X_nans[0]!=0]

X_test_nans = X_test.isna().sum().reset_index()
X_test_nans[X_test_nans[0]!=0]

,index,0


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier

# Assuming you have your data and target variables stored as X and y

# Create the base model
base_model = GradientBoostingClassifier(random_state=42)

# Create the RFE object
rfe = RFE(estimator=base_model, n_features_to_select=7, step=4)

# Fit the RFE to the data
rfe.fit(X, y)

# Get the selected feature indices
selected_features = X.columns[rfe.support_]

# Create new datasets with the selected features
X_rfe = X[selected_features]
X_test_rfe = X_test[selected_features]

# Train your model with the selected features
# model = base_model.fit(X_rfe, y)

# # Make predictions on the test set
# y_pred = model.predict(X_test_rfe)

In [ ]:
selected_features

Index(['head_to_head_win_percentage', 'ratio_top_3_bowlers_avg_econ',
       'ratio_top_3batsmen_avg_sr', 'ground_avg_runs', 'team1only_avg_runs',
       'ratio_last_5_wickets', 'ratio_win_percentage'],
      dtype='object')

In [ ]:
feature_names = X_rfe.columns.tolist()


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_rfe)
X_test_scaled = scaler.transform(X_test_rfe)

In [ ]:
# user-defined parameters

algo_name = 'GradientBoostingClassifier'
is_ensemble = 'no'
n_trees = 10
depth = 2
lr = 0.01
clf_gbm = GradientBoostingClassifier(n_estimators = n_trees, max_depth = depth, learning_rate = lr).fit(X_scaled,y)

In [ ]:
train_merged['y_pred_01'] = clf_gbm.predict(X_scaled)
test_merged['y_pred_01'] = clf_gbm.predict(X_test_scaled)

In [ ]:
print(classification_report(y, clf_gbm.predict(X_scaled), labels=[0,1]))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86       486
           1       0.87      0.84      0.86       489

    accuracy                           0.86       975
   macro avg       0.86      0.86      0.86       975
weighted avg       0.86      0.86      0.86       975



In [ ]:
models = [
    ('GBM', GradientBoostingClassifier(), {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1]
    }),
    ('LightGBM', lgb.LGBMClassifier(random_state=42), {
        'max_depth': [3, 5, 7],
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'reg_alpha': [0.1, 0.3, 0.5],
        'reg_lambda': [0.1, 0.3, 0.5]
    }),
    ('XGBoost', xgb.XGBClassifier(random_state=42), {
        'max_depth': [3, 5, 7],
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'reg_alpha': [0.1, 0.3, 0.5],
        'reg_lambda': [0.1, 0.3, 0.5]
    }),
    ('CatBoost', CatBoostClassifier(random_state=42, verbose=False), {
        'depth': [3, 5, 7],
        'iterations': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1],
        'l2_leaf_reg': [0.1, 0.3, 0.5],
        'bagging_temperature': [0.1, 0.3, 0.5]
    })
]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
# Define the time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Define the models and their hyperparameters
models = [
    ('GBM', GradientBoostingClassifier(random_state=42), {
        'n_estimators': [100, 200, 300, 400],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'min_samples_split': [2, 5, 10]
    }),
    ('LightGBM', lgb.LGBMClassifier(random_state=42), {
        'num_leaves': [31, 63, 127],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'n_estimators': [100, 200, 300, 400],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }),
    ('XGBoost', xgb.XGBClassifier(random_state=42), {
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'n_estimators': [100, 200, 300, 400],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }),
    ('CatBoost', CatBoostClassifier(random_state=42, verbose=False), {
        'depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'iterations': [100, 200, 300, 400],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bylevel': [0.8, 0.9, 1.0]
    })
]

# Perform grid search with time-based cross-validation
best_model = None
best_score = 0
for name, model, params in models:
    grid_search = GridSearchCV(model, params, cv=tscv, scoring='accuracy', n_jobs=-1, verbose=1)
    grid_search.fit(X_scaled, y)
    cv_score = grid_search.best_score_
    print(f"{name} Best Parameters: {grid_search.best_params_}")
    print(f"{name} Cross-Validation Score: {cv_score}")
    if cv_score > best_score:
        best_model = grid_search.best_estimator_
        best_score = cv_score

print(f"\nBest Model: {best_model}")
print(f"Best Model Cross-Validation Score: {best_score}")

# Make predictions using the best model
train_merged['y_pred_01'] = best_model.predict(X_scaled)
test_merged['y_pred_01'] = best_model.predict(X_test_scaled)

# Print classification report
print(classification_report(y, best_model.predict(X_scaled), labels=[0,1]))

Fitting 5 folds for each of 576 candidates, totalling 2880 fits
GBM Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 300, 'subsample': 1.0}
GBM Cross-Validation Score: 0.8691358024691358
Fitting 5 folds for each of 1728 candidates, totalling 8640 fits
[LightGBM] [Info] Number of positive: 489, number of negative: 486
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000137 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1545
[LightGBM] [Info] Number of data points in the train set: 975, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501538 -> initscore=0.006154
[LightGBM] [Info] Start training from score 0.006154
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [ ]:
best_model = None
best_score = 0
for name, model, params in models:
    grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_scaled, y)  # Use X_scaled and y
    cv_score = grid_search.best_score_
    print(f"{name} Best Parameters: {grid_search.best_params_}")
    print(f"{name} Cross-Validation Score: {cv_score}")
    if cv_score > best_score:
        best_model = grid_search.best_estimator_
        best_score = cv_score
print(f"\nBest Model: {best_model}")
print(f"Best Model Cross-Validation Score: {best_score}")

GBM Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
GBM Cross-Validation Score: 0.8860818713450292
[LightGBM] [Info] Number of positive: 480, number of negative: 468
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000141 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2001
[LightGBM] [Info] Number of data points in the train set: 948, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506329 -> initscore=0.025318
[LightGBM] [Info] Start training from score 0.025318
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Define the base models
base_models = [
    ('GBM', GradientBoostingClassifier(random_state=42)),
    ('LightGBM', lgb.LGBMClassifier(random_state=42)),
    ('XGBoost', xgb.XGBClassifier(random_state=42)),
    ('CatBoost', CatBoostClassifier(random_state=42, verbose=False))
]

# Define the meta-model
meta_model = LogisticRegression()

# Create the stacking classifier
stacking_classifier = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

# Define the parameter grid for grid search
param_distributions = {
    'GBM__n_estimators': [50, 100],
    'GBM__max_depth': [3, 5],
    'GBM__learning_rate': [0.05, 0.1],
    'LightGBM__max_depth': [3, 5],
    'LightGBM__n_estimators': [50, 100],
    'LightGBM__learning_rate': [0.05, 0.1],
    'XGBoost__max_depth': [3, 5],
    'XGBoost__n_estimators': [50, 100],
    'XGBoost__learning_rate': [0.05, 0.1],
    'final_estimator__C': [0.1, 1]
}

# Perform grid search
grid_search = GridSearchCV(stacking_classifier, param_distributions, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_scaled, y)

# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

# Use the best model for predictions
best_stacking_model = grid_search.best_estimator_
train_merged['y_pred_01'] = best_stacking_model.predict(X_scaled)
test_merged['y_pred_01'] = best_stacking_model.predict(X_test_scaled)

print(classification_report(y, best_stacking_model.predict(X_scaled), labels=[0,1]))

In [ ]:
train_merged['y_pred_01'] = best_model.predict(X_scaled)
test_merged['y_pred_01'] = best_model.predict(X_test_scaled)

In [ ]:
print(classification_report(y, best_model.predict(X_scaled), labels=[0,1]))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       486
           1       0.92      0.91      0.92       489

    accuracy                           0.92       975
   macro avg       0.92      0.92      0.92       975
weighted avg       0.92      0.92      0.92       975



In [ ]:
train_merged['win_pred_score'] = best_model.predict_proba(X_scaled)[:,1]
test_merged['win_pred_score'] = best_model.predict_proba(X_test_scaled)[:,1]

In [ ]:
train_merged['win_pred_score'] = np.where( (train_merged['y_pred_01']==0), (1-train_merged['win_pred_score']), train_merged['win_pred_score'])
test_merged['win_pred_score'] = np.where( (test_merged['y_pred_01']==0), (1-test_merged['win_pred_score']), test_merged['win_pred_score'])

In [ ]:
train_merged['win_pred_team_id'] = np.where( (train_merged['y_pred_01']==0), (train_merged['team1_id_x']), train_merged['team2_id_x'])
test_merged['win_pred_team_id'] = np.where( (test_merged['y_pred_01']==0), (test_merged['team1_id_x']), test_merged['team2_id_x'])

In [ ]:
import numpy as np

# First, let's check the lengths
print(f"Number of feature names: {len(feature_names)}")
print(f"Number of feature importances: {len(best_model.feature_importances_)}")

# Convert feature importances to float type
feature_importances = np.array(best_model.feature_importances_, dtype=float)

# If they're different, we need to adjust our approach
if len(feature_names) == len(feature_importances):
    df_feat_importance = pd.DataFrame({
        'feat_name': feature_names,
        'model_feat_imp_train': feature_importances
    })
else:
    # If lengths don't match, we'll create a DataFrame with indices and then map feature names
    df_feat_importance = pd.DataFrame({
        'feat_index': range(len(feature_importances)),
        'model_feat_imp_train': feature_importances
    })
    # Map feature names if possible
    if len(feature_names) >= len(feature_importances):
        df_feat_importance['feat_name'] = df_feat_importance['feat_index'].map(lambda i: feature_names[i] if i < len(feature_names) else f'Unknown_{i}')
    else:
        df_feat_importance['feat_name'] = df_feat_importance['feat_index'].map(lambda i: feature_names[i] if i < len(feature_names) else f'Unknown_{i}')

# Sort and display top 10 features
df_feat_importance = df_feat_importance.sort_values(by='model_feat_imp_train', ascending=False).reset_index(drop=True).head(10)
print(df_feat_importance)

# Ensure that model_feat_imp_train is float type
df_feat_importance['model_feat_imp_train'] = df_feat_importance['model_feat_imp_train'].astype(float)

# If you need to save this to a file, you can use:
# df_feat_importance.to_csv('feature_importance.csv', index=False)

Number of feature names: 7
Number of feature importances: 7
                      feat_name  model_feat_imp_train
0   head_to_head_win_percentage             41.873975
1  ratio_top_3_bowlers_avg_econ             28.820539
2     ratio_top_3batsmen_avg_sr             11.733813
3               ground_avg_runs              5.516034
4          ratio_last_5_wickets              4.509587
5          ratio_win_percentage              3.846061
6            team1only_avg_runs              3.699990


In [ ]:
df_feat_importance = pd.DataFrame({'feat_name':X_scaled.columns.tolist(), 'model_feat_imp_train':best_model.feature_importances_}).sort_values(by='model_feat_imp_train', ascending=False)\
                                                                                                                        .reset_index(drop=True).head(10)
df_feat_importance

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
# Get the feature importances for each base model
feat_importances = []
for idx, est in enumerate(best_stacking_model.estimators_):
    est_name = est.__class__.__name__  # Get the name of the estimator class
    if hasattr(est, 'feature_importances_'):
        feat_importances.append(pd.DataFrame({'feat_name': X.columns.tolist(),
                                              'model_feat_imp': est.feature_importances_,
                                              'model_name': est_name}))

# Concatenate the feature importances from all base models
if feat_importances:
    df_feat_importance = pd.concat(feat_importances, ignore_index=True)
    df_feat_importance = df_feat_importance.sort_values(by='model_feat_imp', ascending=False)\
                                            .reset_index(drop=True)\
                                            .head(10)
    print(df_feat_importance)
else:
    print("No feature importances available for the base models.")

NameError: name 'best_stacking_model' is not defined

In [ ]:
df_feat_importance = pd.DataFrame({'feat_name':X_scaled.columns.tolist(), 'model_feat_imp_train':clf_gbm.feature_importances_}).sort_values(by='model_feat_imp_train', ascending=False)\
                                                                                                                        .reset_index(drop=True).head(10)
df_feat_importance

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
df_feat_importance = pd.DataFrame({
    'feat_name': X_rfe.columns.tolist(),
    'model_feat_imp_train': clf_gbm.feature_importances_
}).sort_values(by='model_feat_imp_train', ascending=False) \
  .reset_index(drop=True).head(10)

df_feat_importance

,feat_name,model_feat_imp_train
0,head_to_head_win_percentage,0.659069
1,ratio_top_3_bowlers_avg_econ,0.340931
2,team_count_50runs,0.000000
3,team_winp,0.000000
4,team1only_avg_runs,0.000000
5,team1_winp_team2,0.000000
6,ground_avg_runs,0.000000
7,ratio_top_3batsmen_avg_sr,0.000000
8,venue_toss_advantage_field,0.000000
9,venue_toss_advantage_bat,0.000000


In [ ]:
# Convert feature importances to float
feat_importances = best_model.feature_importances_.astype(float)

# Create the feature importance DataFrame
df_feat_importance = pd.DataFrame({'feat_name': X.columns.tolist(), 'model_feat_imp_train': feat_importances}).sort_values(by='model_feat_imp_train', ascending=False).reset_index(drop=True).head(10)

ValueError: All arrays must be of the same length

In [ ]:
df_feat_importance

,feat_name,model_feat_imp_train
0,head_to_head_win_percentage,41.873975
1,ratio_top_3_bowlers_avg_econ,28.820539
2,ratio_top_3batsmen_avg_sr,11.733813
3,ground_avg_runs,5.516034
4,ratio_last_5_wickets,4.509587
5,ratio_win_percentage,3.846061
6,team1only_avg_runs,3.699990


In [ ]:
train_merged['dataset_type'] = 'train'
test_merged['dataset_type'] = 'r1'

In [ ]:
df_file1['dataset_type'] = 'r2'

In [ ]:
# user-defined parameters

algo_name = 'CatBoostClassifier'
is_ensemble = 'no'
n_trees = 7
depth = 3
lr = 0.05

In [ ]:
# Check for duplicates
duplicate_matches = df_file1['match id'].duplicated().sum()
if duplicate_matches > 0:
    print(f"Warning: {duplicate_matches} duplicate match IDs found in the test set")

In [ ]:
# prompt: delete duplicate as per previous code "duplicate_matches = df_file1['match id'].duplicated().sum()
# if duplicate_matches > 0:
#     print(f"Warning: {duplicate_matches} duplicate match IDs found in the test set")"

df_file1.drop_duplicates(subset='match id', inplace=True)


In [ ]:
df_file1 = test_merged[['match id', 'dataset_type', 'win_pred_team_id', 'win_pred_score']]

# Add the top 10 feature columns
for i, col in enumerate(df_feat_importance['feat_name'].head(10)):
    df_file1[f'indep_feat_id{i+1}'] = test_merged[col]

# Add model information
df_file1['train_algorithm'] = algo_name
df_file1['is_ensemble'] = is_ensemble
df_file1['train_hps_trees'] = n_trees
df_file1['train_hps_depth'] = depth
df_file1['train_hps_lr'] = lr

<ipython-input-95-c3f40e6118f5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_file1[f'indep_feat_id{i+1}'] = test_merged[col]
<ipython-input-95-c3f40e6118f5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_file1[f'indep_feat_id{i+1}'] = test_merged[col]
<ipython-input-95-c3f40e6118f5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [ ]:
duplicate_matches = df_file1['match id'].duplicated().sum()
if duplicate_matches > 0:
    print(f"Warning: {duplicate_matches} duplicate match IDs found in the test set")
    df_file1.drop_duplicates(subset='match id', inplace=True)

In [ ]:
## refactor

df_file1 = pd.concat([test_merged[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))]
                     ])

renaming_dict = {}
for i,col in enumerate(list(df_feat_importance['feat_name'].head(10))):
    renaming_dict[col] = f'indep_feat_id{i+1}'
df_file1.rename(columns=renaming_dict, inplace=True)

for i in range(1,11):
    if f'indep_feat_id{i}' not in df_file1.columns:
        df_file1[f'indep_feat_id{i}'] = np.nan

df_file1['train_algorithm'] = algo_name
df_file1['is_ensemble'] = is_ensemble
df_file1['train_hps_trees'] = n_trees
df_file1['train_hps_depth'] = depth
df_file1['train_hps_lr'] = lr

In [ ]:
df_file1.shape
df_file1.head()

,match id,dataset_type,win_pred_team_id,win_pred_score,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr
0,8887696,r1,33914,0.688427,0.250000,NaN,NaN,0.0,NaN,0.166667,0.000000,CatBoostClassifier,no,7,3,0.05
1,8887738,r1,33935,0.654219,0.571429,NaN,NaN,0.0,NaN,0.543478,0.000000,CatBoostClassifier,no,7,3,0.05
2,8887724,r1,33935,0.953106,1.000000,NaN,NaN,139.1,NaN,0.277778,179.866667,CatBoostClassifier,no,7,3,0.05
3,9027374,r1,18570,0.926499,0.000000,NaN,NaN,0.0,NaN,0.088235,0.000000,CatBoostClassifier,no,7,3,0.05
4,8887815,r1,33935,0.702851,0.666667,NaN,NaN,0.0,NaN,0.416667,179.866667,CatBoostClassifier,no,7,3,0.05


'team_count_50runs', 'team_winp', 'team1only_avg_runs','team1_winp_team2', 'ground_avg_runs', 'ratio_top_3batsmen_avg_sr','ratio_top_3_bowlers_avg_econ', 'venue_toss_advantage_field', 'venue_toss_advantage_bat','head_to_head_win_percentage'

In [ ]:
feature_desc = {'team_count_50runs':'Ratio of number of 50s by players in team1 to number of 50s by players in team2 in last 15 games',\
               'team_winp':'Ratio of team1\'s win % to team2\'s win % in last 5 games',\
               'ground_avg_runs':'average runs scored in the ground in last 15 games',\
               'team1_winp_team2':'Team1\'s win percentage against Team2 in last 15 games',\
               'team1only_avg_runs':'team1\'s avg inning runs in last 15 games',\
               'head_to_head_win_percentage': 'head to head winning percentage',\
               'ratio_top_3_bowlers_avg_econ': 'ratio of top 3 bowler average economy',\
               'ratio_top_3batsmen_avg_sr': 'ratio of top 3 batsmen average',\
                'venue_toss_advantage_field': 'venue specefic win percentage for the team the choose to field',\
                'venue_toss_advantage_bat': 'venue specefic win percentage for the team the choose to bat',\
                'ratio_win_percentage':'ratio of team1 win against team1',\
                'ratio_win_percentage_in_series': 'ratio of win percentage of team1 to team2 in a particular series',\
                'team1_top_3_bowlers_avg_econ': 'team1\'s avg bowler economy in top 3 bowlers in last 15 games',\
                'team2_top_3_bowlers_avg_econ': 'team2\'s avg bowler economy in top 3 bowlers in last 15 games',\
                'team1_top_3_batsmen_avg_sr': 'team1\'s avg batsmen average in top 3 batsmen in last 15 games',\
                'team2_top_3_batsmen_avg_sr': 'team2\'s avg batsmen average in top 3 batsmen in last 15 games',\
                'ratio_last_5_wickets':'ratio of wickets by team1 to team2 in last 5 matches',\
                'ratio_last_5_runs': 'ratio of runs by team1 to team2 in last 5 matches',\
                'ratio_avg_wickets_at_venue': 'ratio of wickets by team1 to team2 at a particular venue',\
                'ratio_avg_runs_at_venue': 'ratio of runs by team1 to team2 at a particular venue',\

                }

In [ ]:
# df_feat_importance.rename(index={0:'feat_id'}, inplace=True)
df_file2 = df_feat_importance
df_file2['feat_id'] = [i+1 for i in df_file2.index]
df_file2['feat_rank_train'] = [i+1 for i in df_file2.index]
df_file2 = df_file2.set_index('feat_id')
df_file2['feat_description'] = df_file2['feat_name'].map(feature_desc)

In [ ]:
# Assuming df_file1 is your submission dataframe and df_feat_importance contains your feature importances

# Ensure all required columns are present
required_columns = ['match id', 'dataset_type', 'win_pred_team_id', 'win_pred_score',
                    'train_algorithm', 'is_ensemble', 'train_hps_trees', 'train_hps_depth', 'train_hps_lr']

# Add independent feature columns
for i in range(1, 11):
    required_columns.append(f'indep_feat_id{i}')

# Create a new dataframe with all required columns
df_submission = pd.DataFrame(columns=required_columns)

# Copy data from df_file1 to df_submission
for col in df_file1.columns:
    if col in df_submission.columns:
        df_submission[col] = df_file1[col]

# Fill in the feature importance columns
for i, feat in enumerate(df_feat_importance['feat_name'].head(10), start=1):
    df_submission[f'indep_feat_id{i}'] = feat

# Fill any remaining independent feature columns with an empty string or a placeholder value
for i in range(1, 11):
    if f'indep_feat_id{i}' not in df_feat_importance['feat_name'].head(10).values:
        df_submission[f'indep_feat_id{i}'] = ''  # or use a placeholder like 'N/A'

# Fill in other required columns if they're missing
df_submission['dataset_type'] = 'r2'
df_submission['train_algorithm'] = algo_name
df_submission['is_ensemble'] = is_ensemble
df_submission['train_hps_trees'] = n_trees
df_submission['train_hps_depth'] = depth
df_submission['train_hps_lr'] = lr

# Ensure all columns are present and in the correct order
df_submission = df_submission[required_columns]

# Check for any missing values
print(df_submission.isnull().sum())

# Save the submission file
df_submission.to_csv('round2_submission.csv', index=False)

match id            0
dataset_type        0
win_pred_team_id    0
win_pred_score      0
train_algorithm     0
is_ensemble         0
train_hps_trees     0
train_hps_depth     0
train_hps_lr        0
indep_feat_id1      0
indep_feat_id2      0
indep_feat_id3      0
indep_feat_id4      0
indep_feat_id5      0
indep_feat_id6      0
indep_feat_id7      0
indep_feat_id8      0
indep_feat_id9      0
indep_feat_id10     0
dtype: int64


In [ ]:
df_file1.to_csv('file1.csv', index=False)
df_file2.to_csv('file2.csv')

In [ ]:
from google.colab import files

files.download('round2_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('file2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>